d ##### Usecase:Get sales data by orders
###### Solutions are given for following problem statements
###### Problem statement-1:Get sales data per day
###### Problem statement-2:Get sales data per month
###### Problem statement-3:Get sales data per year
###### Problem statement-4:Get monthly sales data from third quarter of 2013 and first quarter of 2014 when Max sales >100 
###### Problem statement-5:Get monthly sales data from last quarter of 2013 where soldquantity_permonth > 25000 
###### Problem statement-6:Get daily sales data from fourth quarter of 2013 and second quarter of 2014 where averagesale > 200 and Numberofsales >=600
###### Problem statement-7:Get daily sales data  where soldquantity_perday > 1500
###### Problem statement-8:Get final qurater of 2013 sales data of TOP5 revenue generating order for each month
###### Problem statement-9:Get average revenue per day and all the orders which are more than average.
###### Problem statement-10:Percentage of order items in Order revenue

In [2]:
from pyspark.sql import SQLContext, Row, SparkSession
from pyspark.sql.functions import round,sum,avg,count,substring
from pyspark.sql.types import *
sqlContext = SQLContext(sc)
spark = SparkSession.builder.master('local').appName('getsalesdatabyorder').enableHiveSupport().getOrCreate()
sqlContext.sql("set spark.sql.shuffle.partitions=10")

Out[27]: DataFrame[key: string, value: string]

In [3]:
ordersdf = spark.read.csv("/FileStore/tables/retaildbtext/Orders.txt", sep=',',schema ='order_id int,order_date string,order_customer_id int, order_status string')
ordersdf.registerTempTable("ordersTbl")
ordersdf.show(5,False)

+--------+----------+-----------------+---------------+
order_id|order_date|order_customer_id|order_status |
+--------+----------+-----------------+---------------+
1 |2013-07-25|11599 |CLOSED |
2 |2013-07-25|256 |PENDING_PAYMENT|
3 |2013-07-25|12111 |COMPLETE |
4 |2013-07-25|8827 |CLOSED |
5 |2013-07-25|11318 |COMPLETE |
+--------+----------+-----------------+---------------+
only showing top 5 rows

In [4]:
orderItemsdf = spark.read.csv("/FileStore/tables/retaildbtext/Order_Items.txt", sep=',', schema='order_item_id int, order_item_order_id int, order_item_product_id  int, order_item_quantity int,order_item_subtotal float,order_item_product_price float')
orderItemsdf.registerTempTable("orderItemsTbl")
orderItemsdf.show(5,False)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
1 |1 |957 |1 |299.98 |299.98 |
2 |2 |1073 |1 |199.99 |199.99 |
3 |2 |502 |5 |250.0 |50.0 |
4 |2 |403 |1 |129.99 |129.99 |
5 |4 |897 |2 |49.98 |24.99 |
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
only showing top 5 rows

In [5]:
sqlContext.sql("select * from ordersTbl").show(5,False)


+--------+----------+-----------------+---------------+
order_id|order_date|order_customer_id|order_status |
+--------+----------+-----------------+---------------+
1 |2013-07-25|11599 |CLOSED |
2 |2013-07-25|256 |PENDING_PAYMENT|
3 |2013-07-25|12111 |COMPLETE |
4 |2013-07-25|8827 |CLOSED |
5 |2013-07-25|11318 |COMPLETE |
+--------+----------+-----------------+---------------+
only showing top 5 rows

In [6]:
sqlContext.sql("select * from orderItemsTbl").show(5,False)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
1 |1 |957 |1 |299.98 |299.98 |
2 |2 |1073 |1 |199.99 |199.99 |
3 |2 |502 |5 |250.0 |50.0 |
4 |2 |403 |1 |129.99 |129.99 |
5 |4 |897 |2 |49.98 |24.99 |
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
only showing top 5 rows

###### Problem statement-1:Get sales data per day

In [8]:
SalesDataPerDay = sqlContext.sql('''select cast(otl.order_date as DATE) as orderdate,round(sum(oitl.order_item_subtotal),2) as Totalrevenue, 
                                   min(oitl.order_item_subtotal) as Min_sales, 
                                   max(oitl.order_item_subtotal) as Max_sales, 
                                   count(oitl.order_item_subtotal) as Numberofsales_perday, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_perday 
                                   from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                   group by otl.order_date 
                                   order by otl.order_date''')
SalesDataPerDay.coalesce(1).write.csv("/FileStore/tables/SalesDataPerDay", compression="none", header ='true')
SalesDataPerDay.show(20,False)

+----------+------------+---------+---------+--------------------+---------------+
orderdate |Totalrevenue|Min_sales|Max_sales|Numberofsales_perday|avgsales_perday|
+----------+------------+---------+---------+--------------------+---------------+
2013-07-25|68153.83 |19.98 |599.99 |339 |201.04 |
2013-07-26|136520.17 |9.99 |499.95 |694 |196.71 |
2013-07-27|101074.34 |17.99 |499.95 |503 |200.94 |
2013-07-28|87123.08 |17.99 |499.95 |438 |198.91 |
2013-07-29|137287.09 |17.99 |499.95 |666 |206.14 |
2013-07-30|102745.62 |25.0 |499.95 |540 |190.27 |
2013-07-31|131878.06 |19.99 |499.95 |641 |205.74 |
2013-08-01|129001.62 |15.99 |499.95 |636 |202.83 |
2013-08-02|109347.0 |19.98 |499.95 |558 |195.96 |
2013-08-03|95266.89 |24.99 |499.95 |485 |196.43 |
2013-08-04|90930.98 |9.99 |499.95 |452 |201.17 |
2013-08-05|75882.31 |15.99 |499.95 |392 |193.58 |
2013-08-06|120573.66 |9.99 |499.95 |626 |192.61 |
2013-08-07|103351.39 |17.99 |499.95 |532 |194.27 |
2013-08-08|76501.66 |14.99 |599.99 |401 |190.78 |
2013-08-09|62316.47 |21.99 |499.95 |322 |193.53 |
2013-08-10|129574.8 |27.99 |500.0 |639 |202.78 |
2013-08-11|71149.59 |24.99 |499.95 |347 |205.04 |
2013-08-12|121883.33 |14.99 |499.95 |613 |198.83 |
2013-08-13|39874.52 |24.99 |499.95 |204 |195.46 |
+----------+------------+---------+---------+--------------------+---------------+
only showing top 20 rows

###### Problem statement-2: Get sales data per month

In [10]:
SalesDataPerMonth = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth,round(sum(oitl.order_item_subtotal),2) as Totalrevenue, 
                                   min(oitl.order_item_subtotal) as Min_sales, 
                                   max(oitl.order_item_subtotal) as Max_sales, 
                                   count(oitl.order_item_subtotal) as Numberofsales_permonth, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_permonth 
                                   from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                   group by ordermonth 
                                   order by ordermonth''')
SalesDataPerMonth.coalesce(1).write.csv("/FileStore/tables/SalesDataPerMonth", compression="none", header ='true')
SalesDataPerMonth.show(20,False)

+----------+------------+---------+---------+----------------------+-----------------+
ordermonth|Totalrevenue|Min_sales|Max_sales|Numberofsales_permonth|avgsales_permonth|
+----------+------------+---------+---------+----------------------+-----------------+
2013-07 |764782.2 |9.99 |599.99 |3821 |200.15 |
2013-08 |2828658.75 |9.99 |1999.99 |14268 |198.25 |
2013-09 |2934527.33 |9.99 |1999.99 |14694 |199.71 |
2013-10 |2624600.66 |9.99 |1999.99 |13216 |198.59 |
2013-11 |3168656.09 |9.99 |599.99 |15905 |199.22 |
2013-12 |2932964.33 |9.99 |1999.99 |14729 |199.13 |
2014-01 |2924447.07 |9.99 |1999.99 |14666 |199.4 |
2014-02 |2778663.71 |9.99 |999.99 |14034 |198.0 |
2014-03 |2862492.27 |9.99 |1999.99 |14350 |199.48 |
2014-04 |2807789.85 |9.99 |1999.99 |14192 |197.84 |
2014-05 |2753078.27 |9.99 |1999.99 |13754 |200.17 |
2014-06 |2703463.49 |9.99 |1999.99 |13305 |203.19 |
2014-07 |2238496.56 |9.99 |1999.99 |11264 |198.73 |
+----------+------------+---------+---------+----------------------+-----------------+

###### Problem statement-3: Get sales data per year

In [12]:
SalesDataPerYear = sqlContext.sql('''select substring(otl.order_date,0,4) as orderyear,round(sum(oitl.order_item_subtotal),2) as Totalrevenue, 
                                   min(oitl.order_item_subtotal) as Min_sales, 
                                   max(oitl.order_item_subtotal) as Max_sales, 
                                   count(oitl.order_item_subtotal) as Numberofsales_year, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_peryear 
                                   from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                   group by orderyear 
                                   order by orderyear''')
SalesDataPerYear.coalesce(1).write.csv("/FileStore/tables/SalesDataPerYear", compression="none", header ='true')
SalesDataPerYear.show(20,False)

+---------+-------------+---------+---------+------------------+----------------+
orderyear|Totalrevenue |Min_sales|Max_sales|Numberofsales_year|avgsales_peryear|
+---------+-------------+---------+---------+------------------+----------------+
2013 |1.525418937E7|9.99 |1999.99 |76633 |199.06 |
2014 |1.906843123E7|9.99 |1999.99 |95565 |199.53 |
+---------+-------------+---------+---------+------------------+----------------+

###### Problem statement-4:Get monthly sales data from third quarter of 2013 and first quarter of 2014 when Max sales >100

In [14]:
SalesDataPerMonthquarter = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth,round(sum(oitl.order_item_subtotal),2) as Totalrevenue, 
                                   min(oitl.order_item_subtotal) as Min_sales, 
                                   max(oitl.order_item_subtotal) as Max_sales, 
                                   count(oitl.order_item_subtotal) as Numberofsales_permonth, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_permonth 
                                   from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                   where (CAST(otl.order_date AS DATE)>='2013-07-01' AND CAST(otl.order_date AS DATE)<='2013-09-30') 
                                   OR( CAST(otl.order_date AS DATE)>='2014-01-01'AND CAST(otl.order_date AS DATE)<='2014-03-31')
                                   group by ordermonth 
                                   having Max_sales > 100
                                   order by ordermonth''')
SalesDataPerMonthquarter.coalesce(1).write.csv("/FileStore/tables/SalesDataPerMonthquarter", compression="none", header ='true')
SalesDataPerMonthquarter.show(20,False)

+----------+------------+---------+---------+----------------------+-----------------+
ordermonth|Totalrevenue|Min_sales|Max_sales|Numberofsales_permonth|avgsales_permonth|
+----------+------------+---------+---------+----------------------+-----------------+
2013-07 |764782.2 |9.99 |599.99 |3821 |200.15 |
2013-08 |2828658.75 |9.99 |1999.99 |14268 |198.25 |
2013-09 |2934527.33 |9.99 |1999.99 |14694 |199.71 |
2014-01 |2924447.07 |9.99 |1999.99 |14666 |199.4 |
2014-02 |2778663.71 |9.99 |999.99 |14034 |198.0 |
2014-03 |2862492.27 |9.99 |1999.99 |14350 |199.48 |
+----------+------------+---------+---------+----------------------+-----------------+

###### Problem statement-5:Get monthly sales data from last quarter of 2013 where soldquantity_permonth>25000

In [16]:
SalesDataPerMonthquant = sqlContext.sql('''select substring(otl.order_date,0,7) as ordermonth,round(sum(oitl.order_item_subtotal),2) as Totalrevenue, 
                                   min(oitl.order_item_subtotal) as Min_sales,  
                                   max(oitl.order_item_subtotal) as Max_sales,
                                   count(oitl.order_item_subtotal) as Numberofsales_permonth, 
                                   sum(oitl.order_item_quantity) as  soldquantity_permonth,
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_permonth 
                                   from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id
                                   where CAST(otl.order_date AS DATE)>='2013-10-01' AND CAST(otl.order_date AS DATE)<='2013-12-31'
                                   group by ordermonth
                                   having soldquantity_permonth>25000
                                   order by ordermonth''')
SalesDataPerMonthquant.coalesce(1).write.csv("/FileStore/tables/SalesDataPerMonthquant", compression="none", header ='true')
SalesDataPerMonthquant.show(20,False)

+----------+------------+---------+---------+----------------------+---------------------+-----------------+
ordermonth|Totalrevenue|Min_sales|Max_sales|Numberofsales_permonth|soldquantity_permonth|avgsales_permonth|
+----------+------------+---------+---------+----------------------+---------------------+-----------------+
2013-10 |2624600.66 |9.99 |1999.99 |13216 |28904 |198.59 |
2013-11 |3168656.09 |9.99 |599.99 |15905 |34705 |199.22 |
2013-12 |2932964.33 |9.99 |1999.99 |14729 |31921 |199.13 |
+----------+------------+---------+---------+----------------------+---------------------+-----------------+

###### Problem statement-6:Get daily sales data from fourth quarter of 2013 and second quarter of 2014 where averagesale > 200 and Numberofsales >=600

In [18]:
SalesDataPerdayavgsales = sqlContext.sql('''select substring(otl.order_date,0,10) as orderdate,round(sum(oitl.order_item_subtotal),2) as Totalrevenue, 
                                   min(oitl.order_item_subtotal) as Min_sales, 
                                   max(oitl.order_item_subtotal) as Max_sales, 
                                   count(oitl.order_item_subtotal) as Numberofsales, 
                                   round(avg(oitl.order_item_subtotal),2) as avgsales 
                                   from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                   where (CAST(otl.order_date AS DATE)>='2013-10-01' AND CAST(otl.order_date AS DATE)<='2013-12-31') 
                                   OR( CAST(otl.order_date AS DATE)>='2014-04-01' AND CAST(otl.order_date AS DATE)<='2014-06-30')
                                   group by orderdate 
                                   having avgsales > 200 and Numberofsales >=600
                                   order by orderdate''')
SalesDataPerdayavgsales.coalesce(1).write.csv("/FileStore/tables/SalesDataPerdayavgsales", compression="none", header ='true')
SalesDataPerdayavgsales.show(20,False)

+----------+------------+---------+---------+-------------+--------+
orderdate |Totalrevenue|Min_sales|Max_sales|Numberofsales|avgsales|
+----------+------------+---------+---------+-------------+--------+
2013-10-06|125221.77 |24.99 |499.95 |622 |201.32 |
2013-10-13|145561.99 |9.99 |499.95 |727 |200.22 |
2013-11-01|121736.89 |9.99 |499.95 |606 |200.89 |
2013-11-03|175599.2 |14.99 |499.95 |860 |204.19 |
2013-11-06|124044.93 |14.99 |499.95 |613 |202.36 |
2013-11-14|149271.96 |15.99 |499.95 |743 |200.9 |
2013-11-22|126926.43 |14.99 |499.95 |622 |204.06 |
2013-11-23|128024.84 |24.99 |499.95 |629 |203.54 |
2013-11-29|136296.63 |25.0 |499.95 |676 |201.62 |
2013-11-30|125511.03 |14.99 |499.95 |620 |202.44 |
2013-12-01|126193.28 |15.99 |499.95 |624 |202.23 |
2013-12-07|127089.81 |9.99 |499.95 |634 |200.46 |
2013-12-20|134371.41 |15.99 |499.95 |668 |201.15 |
2014-04-03|127594.93 |9.99 |499.95 |623 |204.81 |
2014-05-06|137077.97 |14.99 |499.95 |676 |202.78 |
2014-05-12|127265.59 |14.99 |499.95 |622 |204.61 |
2014-05-16|139834.47 |19.99 |599.99 |670 |208.71 |
2014-06-08|121925.54 |15.99 |499.95 |606 |201.2 |
2014-06-13|128392.34 |19.98 |499.95 |626 |205.1 |
2014-06-19|147183.83 |19.99 |499.95 |709 |207.59 |
+----------+------------+---------+---------+-------------+--------+
only showing top 20 rows

###### Problem statement-7:Get daily sales data  where soldquantity_perday > 1500

In [20]:
SalesDataPerdaysoldquant = sqlContext.sql('''select substring(otl.order_date,0,10) as orderdate,round(sum(oitl.order_item_subtotal),2) as Totalrevenue, 
                                   min(oitl.order_item_subtotal) as Min_sales,  
                                   max(oitl.order_item_subtotal) as Max_sales,
                                   count(oitl.order_item_subtotal) as Numberofsales_perday, 
                                   sum(oitl.order_item_quantity) as  soldquantity_perday,
                                   round(avg(oitl.order_item_subtotal),2) as avgsales_perday 
                                   from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id
                                   group by orderdate
                                   having soldquantity_perday>1500
                                   order by orderdate''')
SalesDataPerdaysoldquant.coalesce(1).write.csv("/FileStore/tables/SalesDataPerdaysoldquant", compression="none", header ='true')
SalesDataPerdaysoldquant.show(25,False)

+----------+------------+---------+---------+--------------------+-------------------+---------------+
orderdate |Totalrevenue|Min_sales|Max_sales|Numberofsales_perday|soldquantity_perday|avgsales_perday|
+----------+------------+---------+---------+--------------------+-------------------+---------------+
2013-07-26|136520.17 |9.99 |499.95 |694 |1511 |196.71 |
2013-07-29|137287.09 |17.99 |499.95 |666 |1518 |206.14 |
2013-09-14|135308.52 |21.99 |499.95 |681 |1518 |198.69 |
2013-09-25|141775.64 |19.98 |499.95 |730 |1609 |194.21 |
2013-09-27|136537.86 |14.99 |499.95 |671 |1509 |203.48 |
2013-10-04|138079.62 |24.99 |499.95 |709 |1591 |194.75 |
2013-10-13|145561.99 |9.99 |499.95 |727 |1574 |200.22 |
2013-11-03|175599.2 |14.99 |499.95 |860 |1822 |204.19 |
2013-11-05|138528.34 |14.99 |499.95 |704 |1622 |196.77 |
2013-11-14|149271.96 |15.99 |499.95 |743 |1700 |200.9 |
2013-11-24|142234.02 |15.99 |499.95 |726 |1526 |195.91 |
2013-12-26|143012.23 |15.99 |999.99 |734 |1567 |194.84 |
2014-01-05|138550.94 |9.99 |499.95 |704 |1575 |196.81 |
2014-02-01|153561.8 |15.99 |499.95 |757 |1660 |202.86 |
2014-02-19|141857.25 |24.99 |499.95 |713 |1513 |198.96 |
2014-04-27|132838.34 |15.99 |499.95 |687 |1524 |193.36 |
2014-06-19|147183.83 |19.99 |499.95 |709 |1574 |207.59 |
2014-06-27|139952.39 |14.99 |499.95 |692 |1590 |202.24 |
2014-07-15|136902.59 |15.99 |499.95 |704 |1501 |194.46 |
+----------+------------+---------+---------+--------------------+-------------------+---------------+

##### Problem statement-8:Get final qurater of 2013 sales data of TOP5 revenue generating order for each month

In [22]:
SalesDataPermonthquarterly = sqlContext.sql('''select * from(select substring(otl.order_date,0,7) as ordermonth,otl.order_id,round(sum(oitl.order_item_subtotal),2) as Totalrevenue,               
                                   rank() over(partition by substring(otl.order_date,0,7) order by round(sum(oitl.order_item_subtotal),2) desc) ranking_order
                                   from ordersTbl otl join orderItemsTbl oitl on otl.order_id=oitl.order_item_order_id 
                                   where (CAST(otl.order_date AS DATE)>='2013-10-01' AND CAST(otl.order_date AS DATE)<='2013-12-31') 
                                   group by otl.order_id,otl.order_date 
                                   order by ordermonth )
                                   where ranking_order<=5 order by 1,3 desc''')
SalesDataPermonthquarterly.coalesce(1).write.csv("/FileStore/tables/SalesDataPermonthquarterly", compression="none", header ='true')
SalesDataPermonthquarterly.show(20,False)

+----------+--------+------------+-------------+
ordermonth|order_id|Totalrevenue|ranking_order|
+----------+--------+------------+-------------+
2013-10 |68736 |2259.95 |1 |
2013-10 |14539 |1899.9 |2 |
2013-10 |59652 |1799.81 |3 |
2013-10 |13445 |1699.88 |4 |
2013-10 |14775 |1649.8 |5 |
2013-11 |18735 |1819.86 |1 |
2013-11 |17185 |1699.91 |2 |
2013-11 |19754 |1699.81 |3 |
2013-11 |60794 |1649.84 |4 |
2013-11 |17659 |1649.82 |5 |
2013-12 |68766 |2699.9 |1 |
2013-12 |22030 |2039.8 |2 |
2013-12 |23631 |1829.86 |3 |
2013-12 |61289 |1799.89 |4 |
2013-12 |23425 |1759.9 |5 |
+----------+--------+------------+-------------+

###### Problem statement-9:Get average revenue per day and all the orders which are more than average.

In [24]:
SalesDataPerdayavgsales = sqlContext.sql('''SELECT order_date, order_id, order_revenue, avg_revenue FROM 
                                          (SELECT otl.order_date, otl.order_id, sum(oitl.order_item_subtotal) order_revenue, 
                                           AVG(sum(oitl.order_item_subtotal)) OVER (PARTITION BY otl.order_date ) avg_revenue  
                                           from ordersTbl otl join orderItemsTbl oitl WHERE otl.order_id = oitl.order_item_order_id GROUP BY  otl.order_date, otl.order_id) 
                                           WHERE order_revenue >= avg_revenue ORDER BY 1, 3 DESC''')
SalesDataPerdayavgsales.coalesce(1).write.csv("/FileStore/tables/SalesDataPerdayavgsales", compression="none", header ='true')
SalesDataPerdayavgsales.show(20,False)

+----------+--------+------------------+-----------------+
order_date|order_id|order_revenue |avg_revenue |
+----------+--------+------------------+-----------------+
2013-07-25|57779 |1649.800048828125 |587.5330286848134|
2013-07-25|84 |1499.8700256347656|587.5330286848134|
2013-07-25|12 |1299.8700256347656|587.5330286848134|
2013-07-25|73 |1279.6499786376953|587.5330286848134|
2013-07-25|28 |1159.9000129699707|587.5330286848134|
2013-07-25|62 |1149.9400329589844|587.5330286848134|
2013-07-25|57764 |1149.9200286865234|587.5330286848134|
2013-07-25|5 |1129.8600387573242|587.5330286848134|
2013-07-25|57788 |1119.860019683838 |587.5330286848134|
2013-07-25|29 |1109.8500213623047|587.5330286848134|
2013-07-25|99 |1099.9300231933594|587.5330286848134|
2013-07-25|57757 |1099.870018005371 |587.5330286848134|
2013-07-25|57782 |1049.8500061035156|587.5330286848134|
2013-07-25|57763 |1029.9000396728516|587.5330286848134|
2013-07-25|57765 |966.9000091552734 |587.5330286848134|
2013-07-25|15 |925.9100189208984 |587.5330286848134|
2013-07-25|11 |919.7899932861328 |587.5330286848134|
2013-07-25|57759 |909.9700126647949 |587.5330286848134|
2013-07-25|101 |899.9400253295898 |587.5330286848134|
2013-07-25|63 |899.9200210571289 |587.5330286848134|
+----------+--------+------------------+-----------------+
only showing top 20 rows

###### Problem statement-10:Percentage of order items in Order revenue

In [26]:
SalesDataPcntorders = sqlContext.sql('''SELECT order_item_order_id, order_item_id, order_item_subtotal, 
                                            round(SUM(order_item_subtotal) OVER(PARTITION BY order_item_order_id),2) order_revenue , 
                                           (round(order_item_subtotal /SUM(order_item_subtotal) OVER(PARTITION BY order_item_order_id),2)) percentage FROM orderItemsTbl ORDER BY 1''')
SalesDataPcntorders.coalesce(1).write.csv("/FileStore/tables/SalesDataPcntorders", compression="none", header ='true')
SalesDataPcntorders.show(20,False)

+-------------------+-------------+-------------------+-------------+----------+
order_item_order_id|order_item_id|order_item_subtotal|order_revenue|percentage|
+-------------------+-------------+-------------------+-------------+----------+
1 |1 |299.98 |299.98 |1.0 |
2 |3 |250.0 |579.98 |0.43 |
2 |4 |129.99 |579.98 |0.22 |
2 |2 |199.99 |579.98 |0.34 |
4 |6 |299.95 |699.85 |0.43 |
4 |8 |199.92 |699.85 |0.29 |
4 |5 |49.98 |699.85 |0.07 |
4 |7 |150.0 |699.85 |0.21 |
5 |9 |299.98 |1129.86 |0.27 |
5 |13 |129.99 |1129.86 |0.12 |
5 |10 |299.95 |1129.86 |0.27 |
5 |11 |99.96 |1129.86 |0.09 |
5 |12 |299.98 |1129.86 |0.27 |
7 |15 |299.98 |579.92 |0.52 |
7 |16 |79.95 |579.92 |0.14 |
7 |14 |199.99 |579.92 |0.34 |
8 |20 |50.0 |729.84 |0.07 |
8 |18 |299.95 |729.84 |0.41 |
8 |19 |199.92 |729.84 |0.27 |
8 |17 |179.97 |729.84 |0.25 |
+-------------------+-------------+-------------------+-------------+----------+
only showing top 20 rows